In [3]:
!pip install google-cloud-vision mediapipe


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.6/514.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 28.5 MB/s eta 0:00:00


In [42]:
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from google.cloud import vision
import tensorflow.lite as tflite


import os
pct_model = tflite.Interpreter(model_path="/content/llite_model-5486383802206912512_tflite_2025-02-01T21_50_19.528893Z_model.tflite")
pct_model.allocate_tensors()

input_details_pct = pct_model.get_input_details()
output_details_pct = pct_model.get_output_details()

ball_model = tflite.Interpreter(model_path="/content/ball_model-1464669334965059584_tflite_2025-02-02T00_33_00.486881Z_model.tflite")
ball_model.allocate_tensors()

input_details_ball = ball_model.get_input_details()
output_details_ball = ball_model.get_output_details()

input_dtype_pct = input_details_pct[0]['dtype']
input_dtype_ball = input_details_ball[0]['dtype']


def calibrate_position(value):
  if value < 1:
    return -1.938 * 1 / value
  else:
    return 0.04238 * 1 / value


client = vision.ImageAnnotatorClient.from_service_account_file('/content/linear-poet-449516-u8-814faa861860.json')

cap = cv2.VideoCapture("/content/Highlights - 2023 12u Little League World Series Baseball - Pool Game 1 - Riverhead LL vs NPMYAC - Those Miller Boys (720p, h264).mp4")
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter("output_video.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
map_out = cv2.VideoWriter("map_video.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
pose_pitcher_3d_out = cv2.VideoWriter("pose_3d_pitcher_video.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
pose_hitter_3d_out = cv2.VideoWriter("pose_3d_hitter_video.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
pose_catcher_3d_out = cv2.VideoWriter("pose_3d_catcher_video.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
heatmap_out = cv2.VideoWriter("heatmap_video.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
ball_out = cv2.VideoWriter("ball_video.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
image = np.zeros((512, 512, 3), dtype=np.uint8)

src_pts = np.array([[0, 0], [frame_width, 0], [frame_width, frame_height * 0.75], [0, frame_height * 0.75]])
dst_pts = np.array([[0, 0], [27.4, 0], [27.4, 27.4], [0, 27.4]])
H, _ = cv2.findHomography(src_pts, dst_pts)


current_time = 0.0
frame_number = 0
previous_positions_ball = []
previous_positions_bat = []
previous_positions_glove = []
ball_position = []
pose_worlds_pitcher = []
connectors_pitcher = []
pose_worlds_hitter = []
connectors_hitter = []
pose_worlds_catcher = []
connectors_catcher = []

home_plate = (27.4 / 2, 0)
first_base = (27.4, -27.4 / 2)
second_base = (27.4 / 2, -27.4)
third_base = (0, -27.4 / 2)
bases = [home_plate, first_base, second_base, third_base, home_plate]

ball_boundary_box_button = False
ball_line_draw_button = False
ball_side_route_button = False
ball_slowmotion_button = False
bat_boundary_box_button = False
bat_line_draw_button = False
glove_boundary_box_button = False
glove_line_draw_button = False
pitcher_boundary_box_button = False
pitcher_pose_button = False
catcher_pose_button = False
catcher_boundary_box_button = False
hitter_boundary_box_button = False
hitter_pose_button = False
hitter_3d_pose_button = False
catcher_3d_pose_button = False
pitcher_3d_pose_button = False
map_button = True
heatmap_button = False

if pitcher_boundary_box_button or pitcher_pose_button or pitcher_3d_pose_button:
  mp_pose_pitcher = mp.solutions.pose
  mp_drawing_pitcher = mp.solutions.drawing_utils
  pose_pitcher = mp_pose_pitcher.Pose(static_image_mode=False, model_complexity=1)

if catcher_boundary_box_button or catcher_pose_button or catcher_3d_pose_button:
  mp_pose_catcher = mp.solutions.pose
  mp_drawing_catcher = mp.solutions.drawing_utils
  pose_catcher = mp_pose_catcher.Pose(static_image_mode=False, model_complexity=1)

if hitter_boundary_box_button or hitter_pose_button or hitter_3d_pose_button:
  mp_pose_hitter = mp.solutions.pose
  mp_drawing_hitter = mp.solutions.drawing_utils
  pose_hitter = mp_pose_hitter.Pose(static_image_mode=False, model_complexity=1)

all_positions = []
player_positions = []
frame_positions = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_number += 1
    if frame_number >= 30:
      break
    _, encoded_image = cv2.imencode('.png', frame)
    content = encoded_image.tobytes()
    image = vision.Image(content=content)
    response = client.object_localization(image=image)
    results = response.localized_object_annotations

    if ball_boundary_box_button or ball_line_draw_button or ball_side_route_button or ball_slowmotion_button:
      if input_dtype_ball == np.uint8:
          image_input_ball = np.expand_dims(frame, axis=0).astype(np.uint8)
      else:
          image_input_ball = np.expand_dims(frame, axis=0).astype(np.float32) / 255.0


      input_shape = input_details_ball[0]['shape']

      image_input_ball = cv2.resize(image_input_ball[0], (input_shape[2], input_shape[1]))
      image_input_ball = np.expand_dims(image_input_ball, axis=0)

      ball_model.set_tensor(input_details_ball[0]['index'], image_input_ball)
      ball_model.invoke()

      boxes = ball_model.get_tensor(output_details_ball[0]['index'])[0]
      classes = ball_model.get_tensor(output_details_ball[1]['index'])[0]
      scores = ball_model.get_tensor(output_details_ball[2]['index'])[0]


      for box, classs, score in zip(boxes, classes, scores):
          y_min, x_min, y_max, x_max = box
          x_min, y_min, x_max, y_max = int(x_min * frame_width), int(y_min * frame_height), int(x_max * frame_width), int(y_max * frame_height)

          if ball_line_draw_button:
              for i in range(1, len(previous_positions_ball)):
                cv2.line(frame, previous_positions_ball[i - 1], previous_positions_ball[i], (0, 255, 0), 2)

          if score > 0.50:
              x_center_ball = int((x_min + x_max) / 2)
              y_center_ball = int((y_min + y_max) / 2)
              ball_position.append((frame_number + 100, y_center_ball))
              previous_positions_ball.append((x_center_ball, y_center_ball))

              if ball_boundary_box_button:

                label = f"ID: Ball"
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

              if ball_side_route_button:
                for i in range(1, len(ball_position)):
                  cv2.line(image, ball_position[i - 1], ball_position[i], (0, 255, 0), 2)
                  ball_out.write(image)

              if ball_slowmotion_button:
                label = f"MPH: 100, Angle: 12"
                cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                out.write(frame)
                out.write(frame)



    if heatmap_button or map_button:
      for result in results:
        vertices_human = result.bounding_poly.normalized_vertices
        points_human = [(int(v.x * frame_width), int(v.y * frame_height)) for v in vertices_human]
        x_min, y_min, x_max, y_max = points_human[0][0], points_human[0][1], points_human[2][0], points_human[2][1]
        class_id = result.name
        confidence = float(result.score)

        if confidence > 0.50 and class_id == 'Person':
            label = f"ID: {class_id}"
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            area = (x_max - x_min) * (y_max - y_min)
            calibration = area / 6600
            y1_calibrated = y_min + calibrate_position(calibration)
            y2_calibrated = y_max + calibrate_position(calibration)

            center_x_calibrated, center_y_calibrated = (x_min + x_max) / 2, (y1_calibrated + y2_calibrated) / 2

            real_world_coords = cv2.perspectiveTransform(
                np.array([[[center_x_calibrated, center_y_calibrated]]], dtype=np.float32), H
            )
            all_positions.append(real_world_coords[0][0])
            frame_positions.append(real_world_coords[0][0])
      player_positions.append(all_positions)

      if map_button:
        plt.figure(figsize=(8, 8))

        plt.plot([point[0] for point in bases], [point[1] for point in bases], color='green', linestyle='-', linewidth=2, label="Field Lines")
        plt.scatter(
            [pos[0] for pos in frame_positions],
            [-pos[1] for pos in frame_positions],
            color='red', label="Players", s=100, edgecolors='black'
        )
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)

        plt.axis("equal")

        temp_image_map_path = f"/content/temp_frame_map{frame_number}.png"
        plt.savefig(temp_image_map_path)
        plt.close()

        graph_map_frame = cv2.imread(temp_image_map_path)
        resized_graph_map = cv2.resize(graph_map_frame, (frame_width, frame_height))
        map_out.write(resized_graph_map)
        os.remove(temp_image_map_path)

      if heatmap_button:
        if player_positions:
          heatmap_data = np.zeros((35, 35))
          for pos in all_positions:
                x_idx = min(int(pos[0] + 5), 35)
                y_idx = int(pos[1])
                if y_idx > 35 / 2:
                  y_idx = int(-pos[1] - 2)
                else:
                  y_idx = int(-pos[1] - 2)

                heatmap_data[y_idx, x_idx] += 1

        offset = 5
        home_plate = (25 / 2 + offset, offset)
        first_base = (25 + offset, 25 / 2 + offset)
        second_base = (25 / 2 + offset, 25 + offset)
        third_base = (offset, 25 / 2 + offset)
        bases = [home_plate, first_base, second_base, third_base, home_plate]
        norm = mcolors.PowerNorm(gamma=0.3)
        extent = [0, heatmap_data.shape[1], 0, heatmap_data.shape[0]]
        plt.imshow(heatmap_data, cmap='jet', interpolation='gaussian', origin='lower', aspect='equal', norm=norm, extent=extent)
        plt.plot([point[0] for point in bases], [point[1] for point in bases], color='white', linestyle='-', linewidth=2, label="Field Lines")
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
        plt.axis("equal")

        temp_image_heatmap_path = f"/content/temp_frame_heatmap_{frame_number}.png"
        plt.savefig(temp_image_heatmap_path)
        plt.close()

        graph_heatmap_frame = cv2.imread(temp_image_heatmap_path)
        resized_graph_heatmap = cv2.resize(graph_heatmap_frame, (frame_width, frame_height))
        heatmap_out.write(resized_graph_heatmap)
        os.remove(temp_image_heatmap_path)

    if pitcher_boundary_box_button or pitcher_pose_button or catcher_boundary_box_button or catcher_pose_button or hitter_boundary_box_button or hitter_pose_button or catcher_3d_pose_button or hitter_3d_pose_button or pitcher_3d_pose_button:
      if input_dtype_pct == np.uint8:
          image_input = np.expand_dims(frame, axis=0).astype(np.uint8)
      else:
          image_input = np.expand_dims(frame, axis=0).astype(np.float32) / 255.0

      input_shape = input_details_pct[0]['shape']

      image_input = cv2.resize(image_input[0], (input_shape[2], input_shape[1]))
      image_input = np.expand_dims(image_input, axis=0)

      pct_model.set_tensor(input_details_pct[0]['index'], image_input)
      pct_model.invoke()

      boxes = pct_model.get_tensor(output_details_pct[0]['index'])[0]
      classes = pct_model.get_tensor(output_details_pct[1]['index'])[0]
      scores = pct_model.get_tensor(output_details_pct[2]['index'])[0]


      for box, classs, score in zip(boxes, classes, scores):
          y_min, x_min, y_max, x_max = box
          x_min, y_min, x_max, y_max = int(x_min * frame_width), int(y_min * frame_height), int(x_max * frame_width), int(y_max * frame_height)

          if pitcher_boundary_box_button or pitcher_pose_button or pitcher_3d_pose_button:
            if  int(classs) == 1 and score > 0.50:
              if pitcher_boundary_box_button:
                label = f"ID: Pitcher"
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

              if pitcher_pose_button or pitcher_3d_pose_button:
                roi = frame[y_min:y_max, x_min:x_max]
                roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                results_pose_pitcher = pose_pitcher.process(roi_rgb)
                landmarks_pitcher = []
                if results_pose_pitcher.pose_landmarks:
                    mp_drawing_pitcher.draw_landmarks(
                        roi,
                        results_pose_pitcher.pose_landmarks,
                        mp_pose_pitcher.POSE_CONNECTIONS,
                        mp_drawing_pitcher.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                        mp_drawing_pitcher.DrawingSpec(color=(66, 66, 245), thickness=2, circle_radius=2)
                    )

                    if pitcher_3d_pose_button:
                      for landmark in results_pose_pitcher.pose_landmarks.landmark:
                        landmarks_pitcher.append((int(landmark.x * frame_width), int(landmark.y * frame_height),
                                          (landmark.z * frame_width)))

                if pitcher_3d_pose_button:
                  plot_landmarks(frame_number, results_pose_pitcher.pose_world_landmarks, mp_pose_pitcher.POSE_CONNECTIONS)
                  temp_image_pitcher_3d_path = f"plot{frame_number}.png"

                  graph_pitcher_3d_frame = cv2.imread(temp_image_pitcher_3d_path)
                  try:
                    resized_graph_pitcher_3d = cv2.resize(graph_pitcher_3d_frame, (frame_width, frame_height))
                  except:
                    break
                  pose_pitcher_3d_out.write(resized_graph_pitcher_3d)
                  os.remove(temp_image_pitcher_3d_path)
                frame[y_min:y_max, x_min:x_max] = roi


          if catcher_boundary_box_button or catcher_pose_button or catcher_3d_pose_button:
            if  int(classs) == 2 and score > 0.50:
              if catcher_boundary_box_button:
                label = f"ID: Catcher"
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

              if catcher_pose_button or catcher_3d_pose_button:
                roi = frame[y_min:y_max, x_min:x_max]
                roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                results_pose_catcher = pose_catcher.process(roi_rgb)
                landmarks_catcher = []

                if results_pose_catcher.pose_landmarks:
                    mp_drawing_catcher.draw_landmarks(
                        roi,
                        results_pose_catcher.pose_landmarks,
                        mp_pose_catcher.POSE_CONNECTIONS,
                        mp_drawing_catcher.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                        mp_drawing_catcher.DrawingSpec(color=(66, 66, 245), thickness=2, circle_radius=2)
                    )
                    if catcher_3d_pose_button:
                      for landmark in results_pose_catcher.pose_landmarks.landmark:
                        landmarks_catcher.append((int(landmark.x * frame_width), int(landmark.y * frame_height),
                                          (landmark.z * frame_width)))

                if catcher_3d_pose_button:
                  plot_landmarks(frame_number, results_pose_catcher.pose_world_landmarks, mp_pose_catcher.POSE_CONNECTIONS)
                  temp_image_catcher_3d_path = f"plot{frame_number}.png"

                  graph_catcher_3d_frame = cv2.imread(temp_image_catcher_3d_path)
                  try:
                    resized_graph_catcher_3d = cv2.resize(graph_catcher_3d_frame, (frame_width, frame_height))
                  except:
                    break
                  pose_catcher_3d_out.write(resized_graph_catcher_3d)
                  os.remove(temp_image_catcher_3d_path)
                frame[y_min:y_max, x_min:x_max] = roi

          if hitter_boundary_box_button or hitter_pose_button or hitter_3d_pose_button:
            if  int(classs) == 0 and score > 0.50:
              if hitter_boundary_box_button:
                label = f"ID: Hitter"
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

              if hitter_pose_button or hitter_3d_pose_button:
                roi = frame[y_min:y_max, x_min:x_max]
                roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                results_pose_hitter = pose_hitter.process(roi_rgb)
                landmarks_hitter = []

                if results_pose_hitter.pose_landmarks:
                    mp_drawing_hitter.draw_landmarks(
                        roi,
                        results_pose_hitter.pose_landmarks,
                        mp_pose_hitter.POSE_CONNECTIONS,
                        mp_drawing_hitter.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                        mp_drawing_hitter.DrawingSpec(color=(66, 66, 245), thickness=2, circle_radius=2)
                    )
                    if hitter_3d_pose_button:
                      for landmark in results_pose_hitter.pose_landmarks.landmark:
                        landmarks_hitter.append((int(landmark.x * frame_width), int(landmark.y * frame_height),
                                          (landmark.z * frame_width)))


                if hitter_3d_pose_button:
                  plot_landmarks(frame_number, results_pose_hitter.pose_world_landmarks, mp_pose_hitter.POSE_CONNECTIONS)
                  temp_image_hitter_3d_path = f"plot{frame_number}.png"

                  graph_hitter_3d_frame = cv2.imread(temp_image_hitter_3d_path)
                  try:
                    resized_graph_hitter_3d = cv2.resize(graph_hitter_3d_frame, (frame_width, frame_height))
                  except:
                    break
                  pose_hitter_3d_out.write(resized_graph_hitter_3d)
                  os.remove(temp_image_hitter_3d_path)
                frame[y_min:y_max, x_min:x_max] = roi


    if bat_boundary_box_button or bat_line_draw_button:
      for result in results:
        vertices_bat = result.bounding_poly.normalized_vertices
        points_bat = [(int(v.x * frame_width), int(v.y * frame_height)) for v in vertices_bat]
        x_min, y_min, x_max, y_max = points_bat[0][0], points_bat[0][1], points_bat[2][0], points_bat[2][1]
        class_id = result.name
        confidence = float(result.score)
        if bat_line_draw_button:
          for i in range(1, len(previous_positions_bat)):
            cv2.line(frame, previous_positions_bat[i - 1], previous_positions_bat[i], (0, 255, 0), 2)

        if confidence > 0.5 and class_id == "Baseball bat":
          x_center_bat = int((x_min + x_max) / 2)
          y_center_bat = int((y_min + y_max) / 2)
          ball_position.append((frame_number, y_center_bat))
          previous_positions_bat.append((x_center_bat, y_center_bat))

          if bat_boundary_box_button:
            label = f"ID: Bat"
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    if glove_boundary_box_button or glove_line_draw_button:
      for result in results:
        vertices_glove = result.bounding_poly.normalized_vertices
        points_glove = [(int(v.x * frame_width), int(v.y * frame_height)) for v in vertices_glove]
        x_min, y_min, x_max, y_max = points_glove[0][0], points_glove[0][1], points_glove[2][0], points_glove[2][1]
        class_id = result.name
        confidence = float(result.score)

        if glove_line_draw_button:
          for i in range(1, len(previous_positions_glove)):
            cv2.line(frame, previous_positions_glove[i - 1], previous_positions_glove[i], (0, 255, 0), 2)

        if confidence > 0.5 and class_id == "Baseball glove":
          x_center_glove = int((x_min + x_max) / 2)
          y_center_glove = int((y_min + y_max) / 2)
          ball_position.append((frame_number, y_center_glove))
          previous_positions_glove.append((x_center_glove, y_center_glove))

          if glove_boundary_box_button:
            label = f"ID: Glove"
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    current_time += 1 / fps
    out.write(frame)

cap.release()
map_out.release()
out.release()
ball_out.release()
pose_pitcher_3d_out.release()
pose_hitter_3d_out.release()
pose_catcher_3d_out.release()
heatmap_out.release()
cv2.destroyAllWindows()
